# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [58]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [59]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [60]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [166]:
#Método 1
columns = ['CustomerID', 'FirstName', 'LastName', 'ProductID', 'ProductName']
total_qty = data.groupby(columns, as_index = False)['Quantity'].sum()
display(total_qty.sort_values(['CustomerID', 'ProductName']).head(20))

#Método 2
total_qty2 = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity': sum})
display(total_qty2.head(20))

,CustomerID,FirstName,LastName,ProductID,ProductName,Quantity
40,33,Lindsay,Santana,324,Apricots - Dried,1
19,33,Lindsay,Santana,219,Assorted Desserts,1
25,33,Lindsay,Santana,273,Bandage - Flexible Neon,1
26,33,Lindsay,Santana,281,"Bar Mix - Pina Colada, 355 Ml",1
32,33,Lindsay,Santana,300,"Beans - Kidney, Canned",1
56,33,Lindsay,Santana,439,"Beef - Chuck, Boneless",1
28,33,Lindsay,Santana,286,Beef - Prime Rib Aaa,1
54,33,Lindsay,Santana,432,Beer - Original Organic Lager,1
27,33,Lindsay,Santana,284,Beer - Rickards Red,1
7,33,Lindsay,Santana,110,Black Currants,1


Quantity
CustomerID ProductName                              
33         Apricots - Dried                        1
           Assorted Desserts                       1
           Bandage - Flexible Neon                 1
           Bar Mix - Pina Colada, 355 Ml           1
           Beans - Kidney, Canned                  1
           Beef - Chuck, Boneless                  1
           Beef - Prime Rib Aaa                    1
           Beer - Original Organic Lager           1
           Beer - Rickards Red                     1
           Black Currants                          1
           Blackberries                            1
           Bread - Italian Roll With Herbs         1
           Bread Foccacia Whole                    1
           Cake - Box Window 10x10x2.5             1
           Cassis                                  1
           Cheese - Cottage Cheese                 1
           Crab - Imitation Flakes                 1
           Fondant - Icing                         1
           French Pastry - Mini Chocolate          1
           General Purpose Trigger                 1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [98]:
products_pivot = total_qty.pivot_table(index = 'ProductName',
                                       columns = 'CustomerID', values = 'Quantity').fillna(0)
products_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [110]:
similarity_mat = squareform(pdist(products_pivot.T, 'euclidean'))
distances = pd.DataFrame(1/(1 + similarity_mat), 
                         index=products_pivot.columns, columns=products_pivot.columns)

display(distances.head())
display(distances.shape)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


(1000, 1000)

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [119]:
#Obtenemos los 5 pares de clientes más similares entre sí (BONUS)
top5_similar = distances.stack().nlargest(1006)
top5_similar = top5_similar[top5_similar != 1]
display(top5_similar)

CustomerID  CustomerID
1072        3535          0.095841
3535        1072          0.095841
3317        3535          0.093498
3535        3317          0.093498
1072        2503          0.092610
2503        1072          0.092610
dtype: float64

In [134]:
#Ciñéndonos al enunciado, obtenemos los 5 clientes más similares para un cliente específico:
#top5_similar_givenID = distances.nlargest(6, '200')[0:6]
top5_similar_givenID1 = distances[33].sort_values(ascending = False)[1:6].reset_index()
display(top5_similar_givenID1)

#O también
top5_similar_givenID2 = distances.nlargest(6, 33)[33][1:6]
display(top5_similar_givenID2)

,CustomerID,33
0,264,0.087047
1,3535,0.087047
2,3317,0.087047
3,2503,0.085983
4,3305,0.085638


CustomerID
264     0.087047
3317    0.087047
3535    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [137]:
similar_users = top5_similar_givenID1['CustomerID'].tolist()
#Método dos, sin resetear la Serie
#similar_users = top5_similar_givenID2.index.tolist()

data_filtered = total_qty[total_qty['CustomerID'].isin(similar_users)]
display(data_filtered.head(30))

,CustomerID,FirstName,LastName,ProductID,ProductName,Quantity
131,264,Virgil,Small,2,Cookie Chocolate Chip With,1
132,264,Virgil,Small,30,"Cheese - Brie, Triple Creme",1
133,264,Virgil,Small,48,Pecan Raisin - Tarts,1
134,264,Virgil,Small,49,Bacardi Breezer - Tropical,1
135,264,Virgil,Small,59,Pastry - Butterscotch Baked,1
136,264,Virgil,Small,67,Wine - Sogrape Mateus Rose,1
137,264,Virgil,Small,77,Potatoes - Idaho 100 Count,1
138,264,Virgil,Small,85,Cheese - Parmesan Cubes,1
139,264,Virgil,Small,88,Remy Red,1
140,264,Virgil,Small,97,Veal - Inside,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [153]:
new_agg_by_product = data_filtered.groupby('ProductName')['Quantity'].sum().sort_values(ascending = False)
display(new_agg_by_product.reset_index().head(10))

,ProductName,Quantity
0,Butter - Unsalted,3
1,Soup - Campbells Bean Medley,3
2,Towels - Paper / Kraft,3
3,Wine - Ej Gallo Sierra Valley,3
4,"Pepper - Black, Whole",3
5,Wine - Blue Nun Qualitatswein,3
6,Scallops 60/80 Iqf,2
7,Chicken - Soup Base,2
8,Sea Bass - Whole,2
9,Cheese Cloth No 100,2


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [165]:
merged_df = new_agg_by_product.to_frame().merge(products_pivot, left_on = 'ProductName', right_on='ProductName')
merged_df_filtered = merged_df[merged_df[33] == 0]
display(merged_df_filtered.head())
display(merged_df_filtered[['Quantity', 33]].sort_values('Quantity', ascending = False)[0 : 5])

,Quantity,33,200,264,356,412,464,477,639,649,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Butter - Unsalted,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Soup - Campbells Bean Medley,3,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,25.0,50.0,25.0,0.0,0.0,0.0,0.0
Wine - Ej Gallo Sierra Valley,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,25.0,50.0,0.0,25.0,0.0,0.0,0.0,0.0
Wine - Blue Nun Qualitatswein,3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Scallops 60/80 Iqf,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,Quantity,33
ProductName,,
Butter - Unsalted,3,0.0
Wine - Ej Gallo Sierra Valley,3,0.0
Wine - Blue Nun Qualitatswein,3,0.0
Soup - Campbells Bean Medley,3,0.0
Bacardi Breezer - Tropical,2,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [186]:
recoms = {}
ids = total_qty['CustomerID'].unique()
for id in ids[0: 5]: #Para facilitar la visualización , cogemos únicamente los 5 primeros IDs
    top5_similar_givenIDi = distances.nlargest(6, id)[id][1:6]
    similar_users = top5_similar_givenIDi.index.tolist()
    data_filtered = total_qty[total_qty['CustomerID'].isin(similar_users)]
    new_agg_by_product = data_filtered.groupby('ProductName')['Quantity']\
    .sum().sort_values(ascending = False)
    merged_df = new_agg_by_product.to_frame()\
    .merge(products_pivot, left_on = 'ProductName', right_on='ProductName')
    merged_df_filtered = merged_df[merged_df[id] == 0]
    recoms[id] = merged_df_filtered[['Quantity', id]].sort_values('Quantity', ascending = False)[0 : 5].index

display(recoms)

{33: Index(['Butter - Unsalted', 'Wine - Ej Gallo Sierra Valley',
        'Wine - Blue Nun Qualitatswein', 'Soup - Campbells Bean Medley',
        'Bacardi Breezer - Tropical'],
       dtype='object', name='ProductName'),
 200: Index(['Soup - Campbells Bean Medley', 'Bay Leaf', 'Pork - Kidney',
        'Wanton Wrap', 'Muffin - Carrot Individual Wrap'],
       dtype='object', name='ProductName'),
 264: Index(['Fish - Scallops, Cold Smoked', 'Wine - Two Oceans Cabernet',
        'Bread - Italian Roll With Herbs', 'Fondant - Icing',
        'Veal - Inside, Choice'],
       dtype='object', name='ProductName'),
 356: Index(['Butter - Unsalted', 'Veal - Inside, Choice', 'Nut - Chestnuts, Whole',
        'Beets - Candy Cane, Organic', 'Wasabi Powder'],
       dtype='object', name='ProductName'),
 412: Index(['Sauce - Gravy, Au Jus, Mix', 'Olive - Spread Tapenade',
        'Pepper - Black, Whole', 'Wine - Blue Nun Qualitatswein',
        'Sprouts - Baby Pea Tendrils'],
       dtype='object', n

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [187]:
results_recoms = pd.DataFrame(recoms)
display(results_recoms)

,33,200,264,356,412
0,Butter - Unsalted,Soup - Campbells Bean Medley,"Fish - Scallops, Cold Smoked",Butter - Unsalted,"Sauce - Gravy, Au Jus, Mix"
1,Wine - Ej Gallo Sierra Valley,Bay Leaf,Wine - Two Oceans Cabernet,"Veal - Inside, Choice",Olive - Spread Tapenade
2,Wine - Blue Nun Qualitatswein,Pork - Kidney,Bread - Italian Roll With Herbs,"Nut - Chestnuts, Whole","Pepper - Black, Whole"
3,Soup - Campbells Bean Medley,Wanton Wrap,Fondant - Icing,"Beets - Candy Cane, Organic",Wine - Blue Nun Qualitatswein
4,Bacardi Breezer - Tropical,Muffin - Carrot Individual Wrap,"Veal - Inside, Choice",Wasabi Powder,Sprouts - Baby Pea Tendrils


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [189]:
similarity_mat_cb = squareform(pdist(products_pivot.T, 'cityblock'))
distances_cb = pd.DataFrame(1/(1 + similarity_mat_cb), 
                         index=products_pivot.columns, columns=products_pivot.columns)

recoms_cb = {}
for id in ids[0: 5]: #Para facilitar la visualización , cogemos únicamente los 5 primeros IDs
    top5_similar_givenIDi = distances_cb.nlargest(6, id)[id][1:6]
    similar_users = top5_similar_givenIDi.index.tolist()
    data_filtered = total_qty[total_qty['CustomerID'].isin(similar_users)]
    new_agg_by_product = data_filtered.groupby('ProductName')['Quantity']\
    .sum().sort_values(ascending = False)
    merged_df = new_agg_by_product.to_frame()\
    .merge(products_pivot, left_on = 'ProductName', right_on='ProductName')
    merged_df_filtered = merged_df[merged_df[id] == 0]
    recoms_cb[id] = merged_df_filtered[['Quantity', id]].sort_values('Quantity', ascending = False)[0 : 5].index

results_recoms_cb = pd.DataFrame(recoms_cb)
display(results_recoms_cb)

,33,200,264,356,412
0,Wine - Charddonnay Errazuriz,Scampi Tail,Pork - Kidney,Butter - Unsalted,Wine - Blue Nun Qualitatswein
1,"Oranges - Navel, 72",Pork - Kidney,Black Currants,"Veal - Inside, Choice",Soup - Campbells Bean Medley
2,Bay Leaf,Muffin - Carrot Individual Wrap,Bread - Italian Roll With Herbs,"Nut - Chestnuts, Whole",Chocolate - Dark
3,Pecan Raisin - Tarts,Soup - Campbells Bean Medley,Wine - Two Oceans Cabernet,"Beets - Candy Cane, Organic",Wine - Chardonnay South
4,Chocolate - Dark,Bay Leaf,Muffin - Carrot Individual Wrap,Wasabi Powder,"Tart Shells - Sweet, 4"


In [192]:
similarity_mat_cosine = squareform(pdist(products_pivot.T, 'cosine'))
distances_consine = pd.DataFrame(1/(1 + similarity_mat_cosine), 
                         index=products_pivot.columns, columns=products_pivot.columns)

recoms_cosine = {}
for id in ids[0: 5]: #Para facilitar la visualización , cogemos únicamente los 5 primeros IDs
    top5_similar_givenIDi = similarity_mat_cosine.nlargest(6, id)[id][1:6]
    similar_users = top5_similar_givenIDi.index.tolist()
    data_filtered = total_qty[total_qty['CustomerID'].isin(similar_users)]
    new_agg_by_product = data_filtered.groupby('ProductName')['Quantity']\
    .sum().sort_values(ascending = False)
    merged_df = new_agg_by_product.to_frame()\
    .merge(products_pivot, left_on = 'ProductName', right_on='ProductName')
    merged_df_filtered = merged_df[merged_df[id] == 0]
    recoms_cosine[id] = merged_df_filtered[['Quantity', id]].sort_values('Quantity', ascending = False)[0 : 5].index

results_recoms_cosine = pd.DataFrame(recoms_cosine)
display(results_recoms_cosine)

AttributeError: 'numpy.ndarray' object has no attribute 'nlargest'